In [ ]:
#@title Python Libraries
!pip install Flask
!pip install face_recognition
!pip install pyAesCrypt
!mkdir loginlist
!mkdir templates

In [ ]:
#@title Create Your First Data
import pyAesCrypt, os
email = "demo@demo.com"
pyAesCrypt.encryptFile("loginlist/" + email + "_ok.jpg", "loginlist/" + email + "_ok.crypt", "alicangonullu")
os.remove("loginlist/" + email + "_ok.jpg")
#For decrypt : pyAesCrypt.decryptFile("loginlist/" + email + ".crypt", "loginlist/" + email + ".jpg", "alicangonullu")

In [ ]:
#@title Backbone Code
import face_recognition, os, pyAesCrypt
from flask import *
from flask import Flask, render_template, request
import base64

app = Flask(__name__)
secret_key = "alicangonullu" # Secret key for decryption

@app.route('/')
def main():
    return render_template("index.html")

@app.route('/login', methods=["POST"])
def face_login():
    image = str(request.form.get("current_image")) #Base64 image from webcam
    email = str(request.form.get("email")) # Email address from database
    
    fh = open("temp.png", "wb")
    fh.write(base64.decodebytes(image.encode('utf-8'))) #Write your temp face file to temp file
    fh.close()

    pyAesCrypt.decryptFile("loginlist/" + email + ".crypt", "loginlist/" + email + ".jpg", secret_key) # Decrypt your verification photo

    yukle_dogrulanacak_resim = face_recognition.load_image_file("temp.png") # Load your temp face file
    yukle_kayitli_resim = face_recognition.load_image_file("loginlist/" + email + ".jpg") # Load your already loaded verification photo

    dogrulanacak_resim_islendi = face_recognition.face_encodings(yukle_dogrulanacak_resim)[0] # Load your temp face file
    kayitli_veri_islendi = face_recognition.face_encodings(yukle_kayitli_resim)[0] # Load your already loaded verification photo

    netice = face_recognition.compare_faces([kayitli_veri_islendi], dogrulanacak_resim_islendi) # Compare both of them

    if(netice[0]):
        verified = "True" # Verified  
    else:
        verified = "False" # Not Verified

    pyAesCrypt.encryptFile("loginlist/" + email + ".jpg", "loginlist/" + email + ".crypt", secret_key) #Encrypt your verification photo
    os.remove("loginlist/" + email + ".jpg") # Cleanup
    os.remove("temp.png") # Cleanup
    return verified

@app.route('/saveimage')
def save():
    return render_template("save.html")

@app.route('/save', methods=["POST"])
def savefile():
    image = str(request.form.get("current_image")) #Base64 image from webcam
    email = str(request.form.get("email")) # Email address from form
    
    fh = open("loginlist/" + email + ".jpg", "wb")
    fh.write(base64.decodebytes(image.encode('utf-8')))# Save your photo
    fh.close()

    pyAesCrypt.encryptFile("loginlist/" + email + ".jpg", "loginlist/" + email + ".crypt", secret_key) # Decrypt your verification photo
    os.remove("loginlist/" + email + ".jpg") # Delete photo
    return "OK"

app.run(host='0.0.0.0', port=81)

In [ ]:
#@title HTML File (templates/index.html)
<!DOCTYPE html>
<html lang="tr">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="X-UA-Compatible" content="ie=edge">
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.2.3/dist/css/bootstrap.min.css">
    <title>Face Login System</title>
</head>

<body onload="init()">
    <h2>Face Login System | Ali Can Gönüllü</h2>
    <div class="container text-center bordered" style="width:280px">
        <form action="/login" method="post">
            <video onclick="snapshot(this);" width=250 height=250 id="video" controls autoplay></video>
            <br>
            <input type="email" placeholder="Email" name="email" class="form-control form-control-sm text-left">
            <br>
            <input type="hidden" accept="image/png" hidden name="current_image" id="current_image">
            <button onclick="login()" class="btn-dark" value="login">Login </button>
            <br>
            <br>
        </form>
        <br>
        <b>Save</b>
        <form action="/save" method="post">
            <video onclick="snapshot(this);" width=250 height=250 id="video" controls autoplay></video>
            <br>
            <input type="email" placeholder="Email" name="email" class="form-control form-control-sm text-left">
            <br>
            <input type="hidden" accept="image/png" hidden name="current_image" id="current_image">
            <button onclick="login()" class="btn btn-primary" value="login">Login </button>
            <br>
            <br>
        </form>
    </div>
    <canvas id="myCanvas" width="400" height="350" hidden></canvas>
</body>
<script>
    //--------------------
    // GET USER MEDIA CODE
    //--------------------
    navigator.getUserMedia = (navigator.getUserMedia ||
        navigator.webkitGetUserMedia ||
        navigator.mozGetUserMedia ||
        navigator.msGetUserMedia);

    var video;
    var webcamStream;
    if (navigator.getUserMedia) {
        navigator.getUserMedia(

            // constraints
            {
                video: true,
                audio: false
            },

            // successCallback
            function (localMediaStream) {
                video = document.querySelector('video');
                video.srcObject = localMediaStream;
                webcamStream = localMediaStream;
            },

            // errorCallback
            function (err) {
                console.log("The following error occured: " + err);
            }
        );
    } else {
        console.log("getUserMedia not supported");
    }



    var canvas, ctx;

    function init() {
        // Get the canvas and obtain a context for
        // drawing in it
        mcanvas = document.getElementById("myCanvas");
        ctx = mcanvas.getContext('2d');
    }

    function login() {
        // Draws current image from the video element into the canvas
        ctx.drawImage(video, 0, 0, mcanvas.width, mcanvas.height);
        var dataURL = mcanvas.toDataURL('image/png');
        document.getElementById("current_image").value = dataURL.replace("data:image/png;base64,", "");

    }

</script>

</html>

In [ ]:
#@title HTML File (templates/save.html)
<!DOCTYPE html>
<html lang="tr">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="X-UA-Compatible" content="ie=edge">
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.2.3/dist/css/bootstrap.min.css">
    <title>Face Login Save System</title>
</head>

<body onload="init()">
    <center><h2>Face Login Save System | Ali Can Gönüllü</h2></center>
    <div class="container text-center bordered" style="width:280px">
        <form action="/save" method="post">
            <video onclick="snapshot(this);" width=250 height=250 id="video" controls autoplay></video>
            <br>
            <input type="email" placeholder="Email" name="email" class="form-control form-control-sm text-left">
            <br>
            <input type="hidden" accept="image/png" hidden name="current_image" id="current_image">
            <button onclick="login()" class="btn-dark" value="login">Login </button>
            <br>
            <br>
        </form>
    </div>
    <canvas id="myCanvas" width="400" height="350" hidden></canvas>
</body>
<script>
    //--------------------
    // GET USER MEDIA CODE
    //--------------------
    navigator.getUserMedia = (navigator.getUserMedia ||
        navigator.webkitGetUserMedia ||
        navigator.mozGetUserMedia ||
        navigator.msGetUserMedia);

    var video;
    var webcamStream;
    if (navigator.getUserMedia) {
        navigator.getUserMedia(

            // constraints
            {
                video: true,
                audio: false
            },

            // successCallback
            function (localMediaStream) {
                video = document.querySelector('video');
                video.srcObject = localMediaStream;
                webcamStream = localMediaStream;
            },

            // errorCallback
            function (err) {
                console.log("The following error occured: " + err);
            }
        );
    } else {
        console.log("getUserMedia not supported");
    }



    var canvas, ctx;

    function init() {
        // Get the canvas and obtain a context for
        // drawing in it
        mcanvas = document.getElementById("myCanvas");
        ctx = mcanvas.getContext('2d');
    }

    function login() {
        // Draws current image from the video element into the canvas
        ctx.drawImage(video, 0, 0, mcanvas.width, mcanvas.height);
        var dataURL = mcanvas.toDataURL('image/png');
        document.getElementById("current_image").value = dataURL.replace("data:image/png;base64,", "");

    }

</script>

</html>